In [1]:
%load_ext sql
%sql postgres://localhost/ex_election

'Connected: @ex_election'

# Election data

The database for these excercises is called `ex_election`.

The process for obtaining this data is described in [this blog post](https://kiwidamien.github.io/munging-with-multiindices-election-data.html).

This is a copy of presidential election results per state from 1952 to 2016. It is a medium sized dataset (i.e. it is probably difficult to do the queries "by hand", but it isn't large enough to stress test your queries).

If you think you have found an error in the questions below, please open a Github Issue.

## Note on table format

Note that some of these questions would be considerably easier if the election data was _tidy_. At the moment, the rows in the election data take the form
```
 state | democrat_votes | republican_votes | other_votes | year 
-------+----------------+------------------+-------------+------
 AL    |         275075 |           149231 |           0 | 1952
 AR    |         226300 |           177155 |           0 | 1952
 AZ    |         108528 |           152042 |           0 | 1952
 CA    |        2257646 |          3035587 |           0 | 1952
```

A tidy dataset would take the form:
```
 state | votes  |   party    | year 
-------+--------+------------+------
 AL    |      0 | other      | 1952
 AL    | 149231 | republican | 1952
 AL    | 275075 | democrat   | 1952
 AR    | 177155 | republican | 1952
 AR    | 226300 | democrat   | 1952
 AR    |      0 | other      | 1952
 AZ    | 152042 | republican | 1952
 AZ    | 108528 | democrat   | 1952
 AZ    |      0 | other      | 1952
 CA    |      0 | other      | 1952
```

Sadly, we don't get to choose the format of the data. However, you could transform the data to look like this using a VIEW (hint: see the UNION command). For reference, the instructions for creating this view are included in this subdirectory, but you should try making it yourself first.

It is not required to transform your data this way, but you might find some of the queries easier. 

In [2]:
%%sql
CREATE VIEW tidy_election
AS
    SELECT 
        state
        ,year
        ,'democrat' party
        ,democrat_votes votes
    FROM
        election
UNION
    SELECT 
        state
        ,year
        ,'republican' party
        ,republican_votes votes
    FROM
        election
UNION
    SELECT
        state
        ,year
        ,'other' party
        ,other_votes votes
    FROM
        election
;       

 * postgres://localhost/ex_election
(psycopg2.ProgrammingError) relation "tidy_election" already exists
 [SQL: "CREATE VIEW tidy_election\nAS\n    SELECT \n        state\n        ,year\n        ,'democrat' party\n        ,democrat_votes votes\n    FROM\n        election\nUNION\n    SELECT \n        state\n        ,year\n        ,'republican' party\n        ,republican_votes votes\n    FROM\n        election\nUNION\n    SELECT\n        state\n        ,year\n        ,'other' party\n        ,other_votes votes\n    FROM\n        election\n;"] (Background on this error at: http://sqlalche.me/e/f405)


## Questions

1. **How many candidates are in the candidate table for the 2000 election?**

In [3]:
%%sql
SELECT 
    count(candidate)
FROM
    candidate 
WHERE
    year = 2000

 * postgres://localhost/ex_election
1 rows affected.


count
3


In [4]:
%%sql 
SELECT
    candidate
FROM
    candidate 
WHERE
    year = 2000

 * postgres://localhost/ex_election
3 rows affected.


candidate
"Gore, Al"
"Nader, Ralph"
"Bush, George W."


2. **How many candidates are in the candidate table for each election from 1984 to 2016?**

In [5]:
%%sql
SELECT
    year, count(candidate)
FROM
    candidate 
WHERE
    year >= 1984
AND
    year <= 2016
GROUP BY 
    year

 * postgres://localhost/ex_election
9 rows affected.


year,count
1984,2
1988,2
1992,3
1996,3
2000,3
2004,2
2008,2
2012,2
2016,3


3. **For each election from 1984 to 2016, give the party that won the popular vote (i.e. the most votes, not the most electoral college seats)**

In [6]:
%%sql
WITH 
    party_totals AS (
        SELECT
            year
            ,party
            ,sum(votes) total
            ,RANK() OVER(PARTITION BY YEAR ORDER BY SUM(votes) DESC) rank
        FROM
            tidy_election
        WHERE
            year >= 1984
        GROUP BY 
            year, party
)
SELECT
    year, party, total
FROM
    party_totals
WHERE
    rank = 1
;

 * postgres://localhost/ex_election
9 rows affected.


year,party,total
1984,republican,54455472
1988,republican,48886597
1992,democrat,44909806
1996,democrat,47400125
2000,democrat,51009810
2004,republican,62039572
2008,democrat,69499428
2012,democrat,65918507
2016,democrat,65853625


In [7]:
%%sql
/* Alternate version without window functions */
SELECT
    year,
    CASE 
        WHEN sum(democrat_votes) > sum(republican_votes) 
        THEN 'democrat'
    ELSE
        'republican'
    END party,
    greatest(sum(democrat_votes), sum(republican_votes)) total
FROM 
    election
WHERE 
    year >= 1984
GROUP BY
    year
ORDER BY
    year;

 * postgres://localhost/ex_election
9 rows affected.


year,party,total
1984,republican,54455472
1988,republican,48886597
1992,democrat,44909806
1996,democrat,47400125
2000,democrat,51009810
2004,republican,62039572
2008,democrat,69499428
2012,democrat,65918507
2016,democrat,65853625


In [8]:
%%sql
/* See which of the two versions has better 
 * performance
 */
EXPLAIN ANALYZE WITH 
    party_totals AS (
        SELECT
            year
            ,party
            ,sum(votes) total
            ,RANK() OVER(PARTITION BY YEAR ORDER BY SUM(votes) DESC) rank
        FROM
            tidy_election
        WHERE
            year >= 1984
        GROUP BY 
            year, party
)
SELECT
    year, party, total
FROM
    party_totals
WHERE
    rank = 1
;

 * postgres://localhost/ex_election
24 rows affected.


QUERY PLAN
CTE Scan on party_totals (cost=136.19..140.69 rows=1 width=44) (actual time=1.690..1.730 rows=9 loops=1)
Filter: (rank = 1)
Rows Removed by Filter: 18
CTE party_totals
-> WindowAgg (cost=132.19..136.19 rows=200 width=52) (actual time=1.687..1.715 rows=27 loops=1)
-> Sort (cost=132.19..132.69 rows=200 width=44) (actual time=1.680..1.682 rows=27 loops=1)
"Sort Key: election.year, (sum(election.democrat_votes)) DESC"
Sort Method: quicksort Memory: 27kB
-> HashAggregate (cost=122.54..124.54 rows=200 width=44) (actual time=1.652..1.660 rows=27 loops=1)
"Group Key: election.year, ('democrat'::text)"


In [9]:
%%sql
EXPLAIN ANALYZE SELECT
    year,
    CASE 
        WHEN sum(democrat_votes) > sum(republican_votes) 
        THEN 'democrat'
    ELSE
        'republican'
    END party,
    greatest(sum(democrat_votes), sum(republican_votes)) total
FROM 
    election
WHERE 
    year >= 1984
GROUP BY
    year
ORDER BY
    year;

 * postgres://localhost/ex_election
10 rows affected.


QUERY PLAN
Sort (cost=23.09..23.13 rows=17 width=44) (actual time=0.300..0.301 rows=9 loops=1)
Sort Key: year
Sort Method: quicksort Memory: 25kB
-> HashAggregate (cost=22.49..22.74 rows=17 width=44) (actual time=0.293..0.295 rows=9 loops=1)
Group Key: year
-> Seq Scan on election (cost=0.00..16.75 rows=459 width=12) (actual time=0.067..0.166 rows=459 loops=1)
Filter: (year >= 1984)
Rows Removed by Filter: 401
Planning Time: 0.087 ms
Execution Time: 0.395 ms


Window functions have a reputation for being slow, and this is one example.

4. **Extension of previous question: for each election from 1984 to 2016, give the party that won the popular vote and the margin (i.e. the amount that the winning party got over the party that came in second place).** You can assume that the third party votes ("Other") are irrelevant, and just compare Democrats and Republicans.

In [10]:
%%sql
SELECT
    year,
    CASE 
        WHEN sum(democrat_votes) > sum(republican_votes) 
        THEN 'democrat'
    ELSE
        'republican'
    END party,
    GREATEST(SUM(democrat_votes), SUM(republican_votes)) total,
    ABS(SUM(democrat_votes) - SUM(republican_votes)) margin
FROM 
    election
WHERE 
    year >= 1984
GROUP BY
    year;

 * postgres://localhost/ex_election
9 rows affected.


year,party,total,margin
2008,democrat,69499428,9549105
2004,republican,62039572,3012457
1996,democrat,47400125,8201370
2000,democrat,51009810,547398
1984,republican,54455472,16878120
2016,democrat,65853625,2868519
2012,democrat,65918507,4984100
1988,republican,48886597,7077121
1992,democrat,44909806,5805256


5. **Which states have had fewer than 3 democratic victories (i.e. fewer than 3 elections where the democrats got the majority of the votes in that state) since 1952?**

In [11]:
%%sql
SELECT
    state,
    SUM(
        CASE 
        WHEN democrat_votes > republican_votes
        THEN 1
        ELSE 0
        END
    ) dem_victories
FROM
    election
GROUP BY 
    state
HAVING 
    SUM(
        CASE 
        WHEN democrat_votes > republican_votes
        THEN 1
        ELSE 0
        END
    ) < 3
ORDER BY 
    state
;

 * postgres://localhost/ex_election
12 rows affected.


state,dem_victories
AK,1
AZ,1
ID,1
IN,2
KS,1
MT,2
ND,1
NE,1
OK,1
SD,1


6. **Which states have had fewer than 3 republican victories since 1952?**

In [12]:
%%sql
SELECT
    state,
    SUM(
        CASE 
        WHEN republican_votes > democrat_votes
        THEN 1
        ELSE 0
        END
    ) dem_victories
FROM
    election
GROUP BY 
    state
HAVING 
    SUM(
        CASE 
        WHEN republican_votes > democrat_votes
        THEN 1
        ELSE 0
        END
    ) < 3
ORDER BY 
    state
;

 * postgres://localhost/ex_election
2 rows affected.


state,dem_victories
DC,0
HI,2


7. We are interested in measuring the partisanship of the states. We will define a partisan state as one that is consistently won by a single party (either Democrat or Republican) since 1988. For example, since 1988 California has been won by the republicans once, and won by the democrats 7 times. Under this metric, California would be considered "partisan". (Note that if we include elections back to 1952, the republicans have won CA 9 times, and democrats have only won it 8 times).

**Find the states where all of the elections since 1988 (including 1988) have been won by the same party**

In [13]:
%%sql
WITH state_winners AS (
    SELECT DISTINCT
        state,
        CASE
            WHEN republican_votes > democrat_votes
            THEN 'republican'
            ELSE 'democrat'
        END winning_party
FROM
    election
WHERE
    year >= 1988
)
SELECT 
    state, MAX(winning_party) party
FROM 
    state_winners
GROUP BY 
    state
HAVING
    count(winning_party) = 1
ORDER BY
    max(winning_party), state
;

 * postgres://localhost/ex_election
21 rows affected.


state,party
DC,democrat
HI,democrat
MA,democrat
MN,democrat
NY,democrat
OR,democrat
RI,democrat
WA,democrat
AK,republican
AL,republican
